# MNIST meets RND

In this tutorial, we go over how to apply random network distillation to non-standard network architectures, specifically, the convoltutional neural networks required to classify the MNIST dataset.

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


import znrnd as rnd

import tensorflow_datasets as tfds

import numpy as np
from flax import linen as nn
import optax
from plotly.subplots import make_subplots
import plotly.graph_objects as go


KeyboardInterrupt



### Making a data generator

The first thing we need to do is create a data generator for the problem.

In [ ]:
class MNISTGenerator(rnd.data.DataGenerator):
    """
    Data generator for MNIST datasets
    """
    def __init__(self):
        """
        Constructor for the MNIST generator class.
        """
        self.ds_train, self.ds_test = tfds.as_numpy(
            tfds.load(
                'mnist:3.*.*', 
                split=['train[:%d]' % 500, 'test[:%d]' % 500], 
                batch_size=-1
            )
        )
        self.data_pool = self.ds_train['image']
    
    def _process_data(self, data_chunk):
        """
        Flatten the images and one-hot encode the labels.
        """  
        image, label = data_chunk['image'], data_chunk['label']

        samples = image.shape[0]
        image = np.array(np.reshape(image, (samples, -1)), dtype=np.float32)
        image = (image - np.mean(image)) / np.std(image)
        label = np.eye(10)[label]

        return {'image': image, 'label': label}
    
    def plot_image(self, indices: list = None, data_list: list = None):
        """
        Plot a single image from the training dataset.
        
        Parameters
        ----------
        indices : list (None)
        data_list : list (None)
        """
        if indices is not None:
            data_length = len(indices)
            data_source = self.ds_train["image"][indices]
        elif data_list is not None:
            data_length = len(data_list)
            data_source = data_list
        else:
            raise TypeError("No valid data provided")
        
        if data_length <= 4:
            columns = data_length
            rows = 1
        else:
            columns = 4
            rows = int(np.ceil(data_length / 4))
            
        fig = make_subplots(rows=rows, cols=columns)
        
        img_counter = 0
        for i in range(1, rows + 1):
            for j in range(1, columns + 1):
                if indices is not None:
                    data = self.ds_train["image"][img_counter].reshape(28, 28)
                else:
                    data = data_list[img_counter].reshape(28, 28)
                fig.add_trace(go.Heatmap(z=data), row=i, col=j)
                if img_counter == len(data_source) - 1:
                    break
                else:
                    img_counter += 1
                
        fig.show()

In [ ]:
data_generator = MNISTGenerator()

In [ ]:
data_generator.plot_image([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16])

### Define the agent

In [ ]:
class CustomModule(nn.Module):
    """
    Simple CNN module.
    """
    @nn.compact
    def __call__(self, x):
        x = nn.Conv(features=32, kernel_size=(3, 3))(x)
        x = nn.relu(x)
        x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
        x = nn.Conv(features=64, kernel_size=(3, 3))(x)
        x = nn.relu(x)
        x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
        x = x.reshape((x.shape[0], -1))  # flatten
        x = nn.Dense(features=256)(x)
        return x

In [ ]:
target = rnd.models.FlaxModel(
        flax_module=CustomModule(),
        optimizer=optax.adam(learning_rate=0.001),
        loss_fn=rnd.loss_functions.MeanPowerLoss(order=2),
        input_shape=(1, 28, 28, 1),
        training_threshold=0.001
    )

predictor = rnd.models.FlaxModel(
        flax_module=CustomModule(),
        optimizer=optax.adam(learning_rate=0.001),
        loss_fn=rnd.loss_functions.MeanPowerLoss(order=2),
        input_shape=(1, 28, 28, 1),
        training_threshold=0.001
    )

In [ ]:
agent = rnd.RND(
        point_selector=rnd.point_selection.GreedySelection(threshold=0.01),
        distance_metric=rnd.distance_metrics.OrderNDifference(order=2),
        data_generator=data_generator,
        target_network=target,
        predictor_network=predictor,
        tolerance=8,
        target_size=10
    )

In [ ]:
agent.run_rnd(visualize=True)

In [ ]:
data_generator.plot_image(data_list=np.array(agent.target_set))